# Loading data for a Language Model

In [1]:
%install-location $cwd/swift-install
%install-swiftpm-flags -c release
%install '.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))' Batcher ModelSupport Datasets

Installing packages:
	.package(url: "https://github.com/tensorflow/swift-models", .branch("master"))
		Batcher
		ModelSupport
		Datasets
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmpi_wnwxbb/swift-install
/home/sudhirsuman/Desktop/swiftPersonal/usr/bin/swift-build: /home/sudhirsuman/anaconda3/lib/libcurl.so.4: no version information available (required by /home/sudhirsuman/Desktop/swiftPersonal/usr/lib/swift/linux/libFoundationNetworking.so)
/home/sudhirsuman/Desktop/swiftPersonal/usr/bin/swift: /home/sudhirsuman/anaconda3/lib/libuuid.so.1: no version information available (required by /home/sudhirsuman/Desktop/swiftPersonal/usr/bin/swift)
Fetching https://github.com/tensorflow/swift-models
Fetching https://github.com/apple/swift-protobuf.git
Fetching https://github.com/kylef/Commander.git
Fetching https://github.com/kylef/Spectre.git
Cloning https://github.com/kylef/Commander.git
Resolving https://github.com/kylef/Commander.git at 0.9.1
Cloning https://github.com/apple/sw

In [2]:
import TensorFlow
import Batcher
import Foundation
import ModelSupport
import Datasets

## Downloading the data

First thing first, we will need to download the data somewhere. We use `DatasetUtilities` for this. You just need to split the url where the archived file is between the filename, extension and rest of the host url, then specify the folder where you want it downloaded. The function `.downloadResource` will then automatically download the archive (if needed) and inflate it (in the process the folder you specified will be created if it didn't exist).

In [3]:
let cwdURL = URL(fileURLWithPath: FileManager.default.currentDirectoryPath)
let dataFolder = DatasetUtilities.downloadResource(
    filename: "wikitext-2", 
    fileExtension: "tgz",
    remoteRoot: URL(string: "https://s3.amazonaws.com/fast-ai-nlp/")!,
    localStorageDirectory: cwdURL.appendingPathComponent("data/", isDirectory: true)
)

Loading resource: wikitext-2
File does not exist locally at expected path: /home/sudhirsuman/Desktop/swiftPersonal/swift-models/Examples/Notebooks/data/wikitext-2 and must be fetched
Fetching URL: https://s3.amazonaws.com/fast-ai-nlp/wikitext-2.tgz...
Archive saved to: /home/sudhirsuman/Desktop/swiftPersonal/swift-models/Examples/Notebooks/data
Extracting archive...


In [4]:
var trainTexts = try! String(contentsOf: dataFolder.appendingPathComponent("train.csv"), encoding: .utf8)

In [5]:
public func readCSV(in file: URL) -> [String] {
    let rawText = try! String(contentsOf: file, encoding: .utf8)
    var rows = rawText.components(separatedBy: "\"\n\"")
    //Removing the initial "
    rows[0] = String(rows[0].dropFirst())
    //Removing the last "\n
    rows[rows.count - 1] = String(rows[rows.count - 1].suffix(2))
    return rows
}

In [6]:
let trainTexts = readCSV(in: dataFolder.appendingPathComponent("train.csv"))
let validTexts = readCSV(in: dataFolder.appendingPathComponent("test.csv"))

In [7]:
trainTexts[0]

" \n = 2013 – 14 York City F.C. season = \n \n The 2013 – 14 season was the <unk> season of competitive association football and 77th season in the Football League played by York City Football Club , a professional football club based in York , North Yorkshire , England . Their 17th @-@ place finish in 2012 – 13 meant it was their second consecutive season in League Two . The season ran from 1 July 2013 to 30 June 2014 . \n Nigel Worthington , starting his first full season as York manager , made eight permanent summer signings . By the turn of the year York were only above the relegation zone on goal difference , before a 17 @-@ match unbeaten run saw the team finish in seventh @-@ place in the 24 @-@ team 2013 – 14 Football League Two . This meant York qualified for the play @-@ offs , and they were eliminated in the semi @-@ final by Fleetwood Town . York were knocked out of the 2013 – 14 FA Cup , Football League Cup and Football League Trophy in their opening round matches . \n 35 

## From texts to numbers

A model won't be able to train on raw texts like the one above. We will need to convert it into numbers first. To do this, there are two different steps: transforming a text into a list of *words* (called tokens) and then transforming those words in numbers. Those steps are usually called tokenization and numericalization in NLP.

### Tokenization

Tokenizing a text is converting it into a list of meaningful tokens. There are several way to do this:
- character-level tokenization just splits the texts in an array of characters
- word-level tokenization splits the texts by words or punctuation symbols
- subword-level tokenization splits the texts by subwords (particularly useful in languages like Turkish or German where you can build longer words by adding prefixes or suffixes)

While character-level tokenization is pretty straightforward, the two other kinds are a bit trickier. How do you split a word like "don't" for instance, which is actually "do not"? In our case, we don't ahve to worry about that since wikitext-103 has been pre-tokenized, so we can just split on space. (Alternatively, we could train a BPE tokenizer on those texts.)

In [8]:
func easyTokenize(_ text: String) -> [String] {
    return text.components(separatedBy: " ")
}

In [9]:
let trainTokenizedTexts = trainTexts.map(easyTokenize)
let validTokenizedTexts = validTexts.map(easyTokenize)

### Numericalization

One our texts are splits into tokens, we can make a mapping token to unique index and convert them into numbers. We usually try to limit the size of the vocabulary by keeping only the most common tokens, or removing the tokens that are only present more than a given number of times. All tokens that are not part of the vocabulary will be changed to `<unk>` (for unkown).

So first, let's count how many times each token is used in our texts. We also save the length of each text since we will need that later on.

In [10]:
func countTokens(_ texts: [[String]]) -> ([Int], [String:Int]) {
    var counts: [String:Int] = [:]
    var lengths: [Int] = []
    for tokens in texts {
        lengths.append(tokens.count)
        for token in tokens {
            counts[token] = (counts[token] ?? 0) + 1
        }
    }
    return (lengths,counts)
}

We only use the training set to build our vocabulary.

In [11]:
let (trainLengths, trainCounts) = countTokens(trainTokenizedTexts)

Then the following function will create a vocabulary containing all the most frequent words up to `maxCount`, and with a minimum frequency of `minFrequency` (NB: a language model can barely learn anything about words rarely present in the dataset). We return a tuple with the two mappings int to string and string to int (often called itos and stoi in NLP).

In [12]:
func makeVocabulary(
    _ counts: [String:Int], 
    minFrequency: Int = 2, 
    maxCount: Int = 60000) 
-> (itos: [Int:String], stoi: [String:Int]) {
    let withoutSpec = counts.filter { $0.0 != "xxunk" && $0.0 != "xxpad" }
    let sorted = withoutSpec.sorted { $0.1 > $1.1 }
    var itos: [Int:String] = [0:"xxunk", 1:"xxpad"]
    var stoi: [String:Int] = ["xxunk":0, "xxpad":1]
    for (i,x) in sorted.enumerated() {
        if i+2 >= maxCount || x.1 < minFrequency { break }
        itos[i+2] = (x.0)
        stoi[x.0] = i+2
    }
    return (itos: itos, stoi: stoi)
}

Let's use our previous counts to build a vocabulary:

In [13]:
let vocabulary = makeVocabulary(trainCounts)

And then we can use it to numericalize our tokenized texts, let's just check what is the index of the unknown token to use it for words that are our of vocabulary.

In [14]:
vocabulary.stoi["<unk>"]

▿ Optional<Int>
  - some : 6


In [15]:
func numericalize(_ tokens: [String], with stoi: [String:Int]) -> [Int] {
    return tokens.map { stoi[$0] ?? 6 }
}

And we can apply it to all our tokenized texts:

In [16]:
let trainNumericalizedTexts = trainTokenizedTexts.map{ numericalize($0, with: vocabulary.stoi) }
let validNumericalizedTexts = validTokenizedTexts.map{ numericalize($0, with: vocabulary.stoi) }

## Understanding the language model dataset

A language model task is to guess the next word in a stream of texts. When having a list of tokenized and numericalized texts, we usually concatenate them all together in one big stream, separate it in the desired numbers of batches (which are `batchSize` chunks of continuous texts) then read through those `sequenceLength` at a time.

Let's look at an example:

In [17]:
let items = [[0,1,2,3,4],[5,6,7,8,9,10],[11,12,13,14,15,16,17,18],[19,20],[21,22]]

In [18]:
let dataset = LanguageModelDataset(batchSize: 4, sequenceLength: 3, numericalizedTexts: items)

Here our stream is the sequence of integers from 0 to 22. With a batchsize of 4, we split it in four chunks which are:
```
0,1,2,3,4
5,6,7,8,9
10,11,12,13,14
15,16,17,18,19
```
The last three bits of the stream are thrown away because we don't have a round multiple of 4.

Then if read with a sequenceLength of 3, the first batch has for input
```
0,1,2
5,6,7
10,11,12
15,16,17
```
and for target the next words:
```
1,2,3
6,7,8
11,12,13
16,17,18
```

Let's put our dataset in a batcher to check it does all of this for us:

In [19]:
let batcher = Batcher(on: dataset, batchSize: 4)

In [20]:
for x in batcher.sequenced() { print(x) }

TensorPair<Int32, Int32>(first: [[ 0,  1,  2],
 [ 5,  6,  7],
 [10, 11, 12],
 [15, 16, 17]], second: [[ 1,  2,  3],
 [ 6,  7,  8],
 [11, 12, 13],
 [16, 17, 18]])
TensorPair<Int32, Int32>(first: [[ 3,  4],
 [ 8,  9],
 [13, 14],
 [18, 19]], second: [[ 4,  5],
 [ 9, 10],
 [14, 15],
 [19, 20]])


The first batch is as expected, and the second one has only a sequence length of 2 because our big chunks of text have a length of 5 here.

Behind the scenes, `LanguageModelDataset` implements a new collection which has the proper length and subscrit, to return the pair input/target of text (and not the raw texts of varying lengths).

With the shuffle enabled, the texts are shuffled before being concatenated to form the stream. We just need to use `languageModelSample` as a `sampleIndices` function.

In [21]:
let batcher = Batcher(on: dataset, batchSize: 4, shuffle: true, sampleIndices: languageModelSample)

In [22]:
for x in batcher.sequenced() { print(x) }

TensorPair<Int32, Int32>(first: [[21, 22, 19],
 [12, 13, 14],
 [17, 18,  0],
 [ 3,  4,  5]], second: [[22, 19, 20],
 [13, 14, 15],
 [18,  0,  1],
 [ 4,  5,  6]])
TensorPair<Int32, Int32>(first: [[20, 11],
 [15, 16],
 [ 1,  2],
 [ 6,  7]], second: [[11, 12],
 [16, 17],
 [ 2,  3],
 [ 7,  8]])


## Applying it to our texts

We can create a `LanguageModelDataset` from all our text. Since it will need all the lengths of every sample to work, we can provide the array of lengths of each text to speed up the init (if we don't, it will make a pass over the dataset to compute them).

In [23]:
let trainSet = LanguageModelDataset(
    batchSize: 64, 
    sequenceLength: 72, 
    numericalizedTexts: trainNumericalizedTexts, 
    lengths: trainLengths
)

In [24]:
let validSet = LanguageModelDataset(
    batchSize: 64, 
    sequenceLength: 72, 
    numericalizedTexts: validNumericalizedTexts
)

And we can batch our samples:

In [25]:
let trainBatcher = Batcher(on: trainSet, batchSize: 64, numWorkers: 4, 
    shuffle: true, sampleIndices: languageModelSample)
let validBatcher = Batcher(on: validSet, batchSize: 64, numWorkers: 4, 
    sampleIndices: languageModelSample)

In [26]:
let b = trainBatcher.sequenced().first {_ in true}!

To iterate through our batches, we just use `.sequenced()` again. Here let's check we do read through the texts in order by storing the first five batches.

In [27]:
var samples: [TensorPair<Int32,Int32>] = []
for b in trainBatcher.sequenced() {
    //Put the training loop here
    if samples.count < 5 { samples.append(b) }
}

To show one of the lines of our tensor, we will use this function:

In [28]:
func showText(_ x: Tensor<Int32>) -> String {
    var tokens = x.scalars.map { vocabulary.itos[Int($0)]! }
    return tokens.joined(separator: " ")
}

Now let's look at the first row of our first batch:

In [29]:
showText(samples[0].first[0])

" \n = General aviation in the United Kingdom = \n \n General aviation in the United Kingdom has been defined as a civil aircraft operation other than a commercial air transport flight operating to a schedule or military aviation . Although the International Civil Aviation Organization ( ICAO ) excludes any form of remunerated aviation from its definition , some commercial operations are often included within the scope of general aviation"


The targets are just shifted one word to the right:

In [30]:
showText(samples[0].second[0])

"\n = General aviation in the United Kingdom = \n \n General aviation in the United Kingdom has been defined as a civil aircraft operation other than a commercial air transport flight operating to a schedule or military aviation . Although the International Civil Aviation Organization ( ICAO ) excludes any form of remunerated aviation from its definition , some commercial operations are often included within the scope of general aviation ("


In the second sample, we pick up exactly where the first batch stopped:

In [31]:
showText(samples[0].first[0])

" \n = General aviation in the United Kingdom = \n \n General aviation in the United Kingdom has been defined as a civil aircraft operation other than a commercial air transport flight operating to a schedule or military aviation . Although the International Civil Aviation Organization ( ICAO ) excludes any form of remunerated aviation from its definition , some commercial operations are often included within the scope of general aviation"


And this works on any rows:

In [32]:
showText(samples[0].first[1])

"city to other localities in Somalia as well as to neighbouring countries . The capital itself is cut into several grid layouts by an extensive road network , with streets supporting the flow of both <unk> and pedestrian traffic . In October 2013 , major construction began on the 23 kilometer road leading to the airport . <unk> by Somali and Turkish engineers , the upgrade was completed in November and included"


In [33]:
showText(samples[1].first[1])

"lane demarcation . The road construction initiative was part of a larger agreement signed by the Somali and Turkish governments to establish Mogadishu and Istanbul as sister cities , and in the process bring all of Mogadishu \'s roads up to modern standards . Following the treaty , the Turkish International Cooperation and Development Agency ( <unk> ) launched a <unk> cleaning project in conjunction with the municipal cleaning department . The"
